In [14]:
!pip install scikit-learn janome

     --------------------------------------- 19.7/19.7 MB 18.7 MB/s eta 0:00:00


In [15]:
import pandas as pd
import pdb
import requests
import json

static_path=""

In [16]:
df = pd.read_csv("test_gpu.csv")

In [17]:
df

,id,group,querystr,ai_train_label,ai_predict_label,ai_predict_score,ai_predict_label_candidate,ai_predict_score_candidate,ai_predict_candidate
0,fc1de8,36,WO2022014496,RF-D/SAW,RF-D/SAW,5.666085,NaN,NaN,NaN
1,6505c2,36,WO2022014497,RF-D/SAW,RF-D/SAW,5.595006,NaN,NaN,NaN
2,72f8c1,36,JPWO2018154950,RF-D/SAW,RF-D/SAW,6.229929,NaN,NaN,NaN
3,1566c2,36,WO2022173038,RF-D/SAW,RF-D/SAW,5.351918,NaN,NaN,NaN
4,97e19e,36,JP2021190846,RF-D/SAW,RF-D/SAW,5.419605,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4967,3aba53,1,JP2013107344,OTHERS/OTHERX,RF-D/CMFD,0.567510,NaN,NaN,NaN
4968,b5ab78,33,JPWO2007029495,NEWPRD/EOLINC,RF-D/RF-FEM,0.692119,NaN,NaN,NaN
4969,d6e4d2,19,JP2010021411,CAP/HI-CAP,CAP/HI-CAP,0.619054,NaN,NaN,NaN
4970,0a0fab,10,JPWO2007007511,EMC/EMIFIL,FD/AD,0.627821,NaN,NaN,NaN


In [20]:
import pandas as pd
import tqdm as tqdm
import numpy as np
import pdb
from typing import Tuple
from sklearn.preprocessing import LabelEncoder

from time import time
from typing import Tuple
from tqdm import tqdm

from janome.tokenizer import Tokenizer
from collections import defaultdict
import codecs
from sklearn.feature_extraction.text import TfidfVectorizer
from janome.tokenfilter import POSKeepFilter


def tfidf(df,head) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    
    t0 = time()

    patentid_list=[] #特許のID配列
    tfidf_word=[] #TFIDFの語句配列
    tfidf_score=[] #TFIDFのスコア配列
    train_label=[]

    for id,tfidf in zip(df.patent_id,df.tfidf):
        df1 = pd.DataFrame(list(tfidf.items()),columns= ['word','tfidf'])
        df1['tfidf']= df1['tfidf'].astype(float)
       
        #df1 = df1[df1['tfidf']> head ]
        df1 = df1.sort_values(by='tfidf',ascending=False).head(head)
        patentid_list.append(id)
        tfidf_word.append(df1['word'].tolist())
        tfidf_score.append(df1['tfidf'].tolist())
    
    train_label = df['ai_train_label'].tolist()

    unique_word = set([x for row in tfidf_word for x in row])
    unique_idx = {w:i for i,w in enumerate(unique_word)}

    print('number of unique words = {}'.format(len(unique_word)))
    print('number of data lengths = {}'.format(len(tfidf_word)))

    to_unique_idx = np.vectorize(unique_idx.get)
    tfidf_arr = np.zeros((len(tfidf_word),len(unique_word)), dtype='float32')

    for i,(words,scores) in enumerate(zip(tfidf_word,tfidf_score)):
        words = np.array(words)
        scores = np.array(scores,dtype=float)
    
        try:
            idx = to_unique_idx(words)
            tfidf_arr[i,idx] = scores
        except:
            pass

# 変換前のデータ
    train_arr = np.array(train_label)
    le = LabelEncoder()
    le.fit(train_arr)
    label_names = le.classes_

    waist_time = time() - t0
    print("tfidf time: %0.3fs" % waist_time)

    unique_words = np.array(list(unique_word))

    return tfidf_arr, train_arr, label_names , unique_words


def get_token(text):
    t = Tokenizer()
    tokens = t.tokenize(text)
    
    word = ""
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
        if part_of_speech == "名詞":
            word +=token.surface + " "
        if part_of_speech == "動詞":
            word +=token.base_form+ " "
        if part_of_speech == "形容詞":
            word +=token.base_form+ " "
        if part_of_speech == "形容動詞":
            word +=token.base_form+ " "
    return word


def tfidf_calc(target,token,max_word):
    corpus=[]
    
    for item in tqdm(target): 
        token=get_token(item)
        corpus.append(token)

    vectorize = TfidfVectorizer(use_idf=True,max_df=0.9)
    
    tfidf = vectorize.fit_transform(corpus)
    
    output =[]
    words = vectorize.get_feature_names()
    for doc_id,vec in zip(range(len(corpus)),tfidf.toarray()):
        jstring=""
        for w_id,tfidf in sorted(enumerate(vec),key=lambda x:x[1],reverse=True)[:max_word]:
            lemma= words[w_id]
            jstring += '"{}"=>"{:.7g}",'.format(lemma,tfidf*100)
        output.append(jstring)
    
    return output


In [21]:
def post_patent_list(body):
    if body:
        data = body['patent']
        tfidf_max_length= body['tfidf_max_length']
        #numofword = body['numofword']
        df = pd.read_json(data)##CrossValidationで使うためにデータフレームにする
        patent_list = df['querystr'].tolist()##クエリストリングをリストにする

        item_patent = cruds.search_patents(
            db=db, mode=mode, patent_numbers=patent_list)
        # modelスケマからjson→pandasに変換する
        json_item_data = jsonable_encoder(item_patent)
        df_s = pd.DataFrame(json_item_data)
        
        for index,row in df_s.iterrows():
            label = df[df.querystr == row.koukai_number].ai_train_label
            df_s.at[index,'ai_train_label'] = label.values[0]
            
        # pickleにつける
        df_s.to_pickle(static_path + body['token'])

       # リダイレクト
        return "/train_tfidf/?token={}&tfidf_max_length={}&numofword={}".format(body['token'], tfidf_max_length,numofword)

staric_path= ""
    
def post_train_tfidf(token, tfidf_max_length):
    unpickled_df = pd.read_pickle(static_path + token)
    tfidf_arr, label_arr,label_names ,unique_word= tfidf(unpickled_df, tfidf_max_length)
    # pickleにつける

    np.save(static_path +'tfidf_arr.npy', tfidf_arr)
    np.save(static_path +'label_arr.npy', label_arr)
    np.save(static_path +'label_names.npy', label_names)
    np.save(static_path + 'unique_names.npy', unique_word)
    # リダイレクト
    return {"token": token, "tfidf_arr": tfidf_arr.shape, "label_arr": label_arr.shape, "label_names": label_names.shape, "unique_word": unique_word.shape}


In [22]:
#絞り込み後の特許リストで交差検証を実施するルーチン
def mlFlow(df,token,tfidf_target,num):
    url ="http://172.26.106.79:7999/patent_list2"
#JSON形式のデータ
    json_data = {
    'patent': df.to_json(),
    'token':token,
    'tfidf_max_length':tfidf_target
    }    
    
    with open('patent_number.json', 'w') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

    res = requests.post( url,  json = json_data)   #dataを指定する    
    json_item_data = res.json()
    
    return json_item_data

In [23]:
#df

In [30]:
#json_data = mlFlow(df,"demo",200,5) #patent number

In [32]:
with open('patent_number.json', 'r') as f:
    json_data = json.load(f)

df_s = pd.DataFrame(json_data)

for index,row in df_s.iterrows():
    label = df[df.querystr == row.koukai_number].ai_train_label
    df_s.at[index,'ai_train_label'] = label.values[0]
            
        # pickleにつける
df_s.to_pickle(static_path + "demo")

In [34]:
#df_s

In [35]:
result = post_train_tfidf("demo",200) #tfidf dataframe

number of unique words = 58436
number of data lengths = 4972
tfidf time: 4.917s


In [42]:
!pip install flask sqlalchemy

     ---------------------------------------- 1.6/1.6 MB 9.1 MB/s eta 0:00:00
     -------------------------------------- 190.7/190.7 kB 1.6 MB/s eta 0:00:00


In [43]:

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,LeaveOneOut, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import Perceptron
from time import time
from tqdm import tqdm
from urllib3 import HTTPResponse
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
from sklearn.ensemble import RandomForestClassifier
from fastapi import APIRouter, Depends, File, HTTPException, Query, UploadFile
from fastapi.encoders import jsonable_encoder
from fastapi.responses import StreamingResponse
from flask import jsonify
from sklearn.feature_extraction import img_to_graph
from sqlalchemy.orm import Session
import requests
from fastapi.responses import RedirectResponse


In [44]:
def get_mlflow(
            token: str = "",numofword: int = 100,
            cvmode: str ="KFold",
            nsplit: int = 5,
            n_candidate: int = 5
            ):

    
    tfidf_arr=np.load(static_path +'tfidf_arr.npy')
    label_arr=np.load(static_path +'label_arr.npy')
    label_names=np.load(static_path +'label_names.npy')
    unique_word = np.load(static_path +'unique_names.npy')

    if cvmode == "KFold":# kfoldでクロスバリデーションを行う
        CrossVaild = StratifiedKFold(n_splits=nsplit, shuffle=True)#,random_state=42)
    elif cvmode == "LeaveOneOut":# 1 vs 1 でクロスバリデーションを行う
        CrossVaild = LeaveOneOut()

    #predict-proba が使えるモデルを使う
    clf =  LogisticRegression(class_weight='balanced',max_iter=300)

    ext = []
    mlresult =[]
    mlresult2=[]

    for train_index, test_index in tqdm(CrossVaild.split(tfidf_arr, label_arr)):
        x_train, x_test = tfidf_arr[train_index], tfidf_arr[test_index]
        y_train, y_test = label_arr[train_index], label_arr[test_index]

    #ms = StandardScaler()
        ms = MinMaxScaler()
        X_train = ms.fit_transform(x_train)
        X_test = ms.transform(x_test)

        clf,ostr = benchmark(clf,X_train, X_test, y_train, y_test)

#def print_top10(feature_names, clf, class_labels):
        if hasattr(clf, "coef_"):
            #ostr["dimensionality"] = "%d" % clf.coef_.shape[1] 
            #ostr["density"] = "%f" % density(clf.coef_) 
            ostr["top10"] = print_top10(numofword,unique_word, clf, label_names)
            intercept = clf.intercept_.tolist()
            mlresult.append(ostr)
            mlresult2.append( dict(zip(label_names,intercept))) #mlresult.append(ostr)
        
        pred_proba_train = clf.predict_proba(X_train)
        pred_proba_test = clf.predict_proba(X_test)

        df = pd.read_pickle(static_path + token)
 
        labels = set()   # 空のセットを生成(set([]))
        for i,value in enumerate(test_index):
            labels.add( df.loc[value].ai_train_label)
        
        labels = list(labels) #listに戻す

        #pdb.set_trace()

        for ind, value in enumerate(test_index):
            output = {"id": str(uuid.uuid4())[:6]}
            output.update({"group": len(mlresult)-1})  #labels.index( df.loc[value].ai_train_label)})
            num = df.loc[value].koukai_number
            output.update({"querystr": num})
            output.update({"ai_train_label": df.loc[value].ai_train_label})
            output.update(
            {"ai_predict_label(pf)": df.loc[value].ai_predict_label})
            output.update(
            {"ai_predict_score(pf)": df.loc[value].ai_predict_score})
            
            ds = {key: val for key, val in zip(label_names, map(
            '{:f}'.format, pred_proba_test[ind].tolist()))}
            #pdb.set_trace()
            max_kv = max(ds.items(), key=lambda x: x[1])
            output.update({"ai_predict_label": max_kv[0]})
            output.update({"ai_predict_score": max_kv[1]})
            ds_sel = dict( sorted(ds.items(),key=lambda x:x[1],reverse=True)[0:n_candidate])
            output.update({"ai_predict_candidate": json.dumps(ds_sel)})
            
            #pdb.set_trace()
            
            ext.append(output)

    return {"crossvalid":ext,"mlresult":mlresult,"mlresult2":mlresult2}


In [45]:
def mlFlow2(token,num_word,cvmode,nsplit,n_candidate):
    url = 'http://172.26.106.79:7999/calc_crossvalid/'
        
    if cvmode=="LeaveOneOut":
        nsplit=2
        
    payload  ={
     'token':token,
     'numofword':num_word,
     'cvmode':cvmode,
     'nsplit':nsplit,
     'n_candidate':n_candidate
    }

   #get送信
    
    res = get_mlflow(token, num_word, cvmode, nsplit , n_candidate)  #requests.get( url,  params=payload)   #dataを指定する    )

    json_item_data = res
        
    #戻り値はJSON形式　dataframeと、mlresultの２つのデータ保存する
    df=json_item_data["crossvalid"]
    mlresult = json_item_data["mlresult"]
    mlresult2 = json_item_data["mlresult2"]

    cust_df = pd.json_normalize(df)
    stjson = cust_df.to_dict("records") 

    return stjson,mlresult,mlresult2
    
    

In [46]:
from typing import Tuple
import numpy as np
from sklearn import metrics
from time import time
import pdb
from typing import Tuple
from sklearn.metrics import confusion_matrix
import uuid

def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."

#print_top10(unique_word, clf, label_names)
def print_top10(topN,feature_names, clf, class_labels):
    
    
    restr = {}

    if len(class_labels)>2:
        N= -1 * topN
        for i, class_label in enumerate(class_labels):
            
            top10 = np.argsort(clf.coef_[i])[N:]
            top10 = top10[::-1]
            top10_vec = np.sort(clf.coef_[i])[N:]
            top10_vec = top10_vec[::-1]
            dic = {key: val for key, val in zip(feature_names[top10],top10_vec)}
            restr[class_label] = dic
            
            #print("{} {} {}".format( class_label,len(top10),len(top10_vec)))

        return restr
    else:
        N= -1 * topN
        
        top10 = np.argsort(clf.coef_[0])[N:]
        top10 = top10[::-1]
        top10_vec = np.sort(clf.coef_[0])[N:]
        top10_vec = top10_vec[::-1]
        dic = {key: val for key, val in zip(feature_names[top10],top10_vec)}
        restr[class_labels[1]] = dic
        #print("{} {} {}".format( class_labels[0],len(top10),len(top10_vec)))

        top10 = np.argsort(clf.coef_[0])[:topN]
        top10_vec = np.sort(clf.coef_[0])[:topN]
        dic = {key: val * -1 for key, val in zip(feature_names[top10],top10_vec)}
        restr[class_labels[0]] = dic
        #print("{} {} {}".format( class_labels[1],len(top10),len(top10_vec)))

        return restr

def benchmark(clf,X_train, X_test, y_train, y_test):
    outstr = {}

    t0 = time()
    clf.fit(X_train, y_train)
    
    train_time = time() - t0
    outstr["train time"] = "%0.3f" % train_time 

    t0 = time()
    pred = clf.predict(X_test)

    #print(confusion_matrix(y_test, pred))

    test_time = time() - t0
    outstr["test time"] = "%0.3f" % test_time 
    
#正解率
    score = metrics.accuracy_score(y_test, pred)
    outstr["accuracy"] = "%0.3f" % score 

#再現性
    recall = metrics.recall_score(y_test, pred, average='macro')
    outstr["recall"] = "%0.3f" % recall

    return clf,outstr


In [ ]:
crossvalid, result, result2 = mlFlow2(token="demo",num_word=5,cvmode="KFold",nsplit = 5,n_candidate = 5) #token,num_word,cvmode,nsplit,n_candidate)

4it [06:30, 99.04s/it]

In [ ]:
[result[x]['train time'] for x in range(5)]

# GPU

In [76]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [86]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils


from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation
from keras.regularizers import l2


def classification_model(output_dim,input_dim):
    model = Sequential()
    model.add(Dense(output_dim, input_shape=(input_dim,), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model


def benchmark_keras(clf,X_train, X_test, y_train, y_test,epochs=500, verbose=0):
    outstr = {}

    t0 = time()
    clf.fit(X_train, y_train,epochs, verbose)
    
    pdb.set_trace()
    
    train_time = time() - t0
    outstr["train time"] = "%0.3f" % train_time 

    t0 = time()
    pred = clf.predict(X_test)

    #print(confusion_matrix(y_test, pred))

    test_time = time() - t0
    outstr["test time"] = "%0.3f" % test_time 
    
#正解率
    score = metrics.accuracy_score(y_test, pred)
    outstr["accuracy"] = "%0.3f" % score 

#再現性
    recall = metrics.recall_score(y_test, pred, average='macro')
    outstr["recall"] = "%0.3f" % recall

    return clf,outstr


In [93]:
def get_mlflow_gpu(
            token: str = "",numofword: int = 100,
            cvmode: str ="KFold",
            nsplit: int = 5,
            n_candidate: int = 5
            ):

    tfidf_arr=np.load(static_path +'tfidf_arr.npy').astype('float32')
    label_arr=np.load(static_path +'label_arr.npy').astype('float32')
    label_names=np.load(static_path +'label_names.npy').astype('float32')
    unique_word = np.load(static_path +'unique_names.npy').astype('float32')

    if cvmode == "KFold":# kfoldでクロスバリデーションを行う
        CrossVaild = StratifiedKFold(n_splits=nsplit, shuffle=True)#,random_state=42)
    elif cvmode == "LeaveOneOut":# 1 vs 1 でクロスバリデーションを行う
        CrossVaild = LeaveOneOut()

    #predict-proba が使えるモデルを使う
    
    #training
    clf = classification_model(
input_dim = len(tfidf_arr),
output_dim = len(label_names)# 3 possible outputs
    )
    #keras_model.fit(X_train, Y_train, epochs=500, verbose=0)
    #clf =  LogisticRegression(class_weight='balanced',max_iter=300)

    ext = []
    mlresult =[]
    mlresult2=[]


    for train_index, test_index in tqdm(CrossVaild.split(tfidf_arr, label_arr)):
        x_train, x_test = tfidf_arr[train_index], tfidf_arr[test_index]
        y_train, y_test = label_arr[train_index], label_arr[test_index]

    #ms = StandardScaler()
        ms = MinMaxScaler()
        X_train = ms.fit_transform(x_train)
        X_test = ms.transform(x_test)

        clf,ostr = benchmark_keras(clf,X_train, X_test, y_train, y_test, epochs=500, verbose=0)

#def print_top10(feature_names, clf, class_labels):
        if hasattr(clf, "coef_"):
            #ostr["dimensionality"] = "%d" % clf.coef_.shape[1] 
            #ostr["density"] = "%f" % density(clf.coef_) 
            ostr["top10"] = print_top10(numofword,unique_word, clf, label_names)
            intercept = clf.intercept_.tolist()
            mlresult.append(ostr)
            mlresult2.append( dict(zip(label_names,intercept))) #mlresult.append(ostr)
        
        pred_proba_train = clf.predict_proba(X_train)
        pred_proba_test = clf.predict_proba(X_test)

        df = pd.read_pickle(static_path + token)
 
        labels = set()   # 空のセットを生成(set([]))
        for i,value in enumerate(test_index):
            labels.add( df.loc[value].ai_train_label)
        
        labels = list(labels) #listに戻す

        #pdb.set_trace()

        for ind, value in enumerate(test_index):
            output = {"id": str(uuid.uuid4())[:6]}
            output.update({"group": len(mlresult)-1})  #labels.index( df.loc[value].ai_train_label)})
            num = df.loc[value].koukai_number
            output.update({"querystr": num})
            output.update({"ai_train_label": df.loc[value].ai_train_label})
            output.update(
            {"ai_predict_label(pf)": df.loc[value].ai_predict_label})
            output.update(
            {"ai_predict_score(pf)": df.loc[value].ai_predict_score})
            
            ds = {key: val for key, val in zip(label_names, map(
            '{:f}'.format, pred_proba_test[ind].tolist()))}
            #pdb.set_trace()
            max_kv = max(ds.items(), key=lambda x: x[1])
            output.update({"ai_predict_label": max_kv[0]})
            output.update({"ai_predict_score": max_kv[1]})
            ds_sel = dict( sorted(ds.items(),key=lambda x:x[1],reverse=True)[0:n_candidate])
            output.update({"ai_predict_candidate": json.dumps(ds_sel)})
            
            #pdb.set_trace()
            
            ext.append(output)

    return {"crossvalid":ext,"mlresult":mlresult,"mlresult2":mlresult2}


In [94]:
def mlFlow3(token,num_word,cvmode,nsplit,n_candidate):
    url = 'http://172.26.106.79:7999/calc_crossvalid/'
        
    if cvmode=="LeaveOneOut":
        nsplit=2
        
    payload  ={
     'token':token,
     'numofword':num_word,
     'cvmode':cvmode,
     'nsplit':nsplit,
     'n_candidate':n_candidate
    }

   #get送信
    
    res = get_mlflow_gpu(token, num_word, cvmode, nsplit , n_candidate)  #requests.get( url,  params=payload)   #dataを指定する    )

    json_item_data = res
        
    #戻り値はJSON形式　dataframeと、mlresultの２つのデータ保存する
    df=json_item_data["crossvalid"]
    mlresult = json_item_data["mlresult"]
    mlresult2 = json_item_data["mlresult2"]

    cust_df = pd.json_normalize(df)
    stjson = cust_df.to_dict("records") 

    return stjson,mlresult,mlresult2

In [95]:
crossvalid, result, result2 = mlFlow3(token="demo",num_word=5,cvmode="KFold",nsplit = 5,n_candidate = 5)

MemoryError: Unable to allocate 2.16 GiB for an array with shape (290543792,) and data type float64